In [1]:
import json
import numpy as np
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle

from ipywidgets import interact, fixed
import ipywidgets as widgets

In [2]:
eps=10
colors = cm.tab10(range(10))
workdir = '/mnt/data5'

In [3]:
df = pd.read_csv(f'{workdir}/output/count/gi-infection/C.diff/C1/outs/spatial/tissue_positions_list.csv', header=None, names=['barcode', 'inside', 'row', 'col', 'y', 'x'])
grid = df[(df['row'].isin([0,1,2])) & (df['col'].isin([1,2,3,4,5]))].sort_values(['row', 'col'])
grid['y'] -= grid.loc[3, 'y']
grid['x'] -= grid.loc[3, 'x']
grid = grid[['x', 'y']].to_numpy()

In [4]:
with open(f'{workdir}/output/count/gi-infection/C.diff/C1/outs/spatial/scalefactors_json.json') as f:
    scales = json.load(f)
radius = scales['spot_diameter_fullres'] / 2

In [5]:
def draw_spot_grid(grid, spot_radius, theta, tile_size):
    rot_mat = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)],
    ])
    lims = [
        min(grid[:, 0].min()-radius-eps, grid[:, 1].min()-radius-eps),
        max(grid[:, 0].max()+radius+eps, grid[:, 1].max()+radius+eps),
    ]
    grid = grid @ rot_mat.T
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set(
        xlim=lims,
        ylim=lims,
    )
    for (x, y), color in zip(grid, colors):
        circle = Circle(xy=(x, y), radius=int(spot_radius), color=color)
        ax.add_patch(circle)
    for (x, y), color in zip(grid, colors):
        tile = Rectangle(xy=(x-tile_size/2, y-tile_size/2), width=tile_size, height=tile_size, fill=None, color=color, lw=5)
        ax.add_patch(tile)
    plt.show()

In [6]:
interact(
    draw_spot_grid,
    grid=fixed(grid),
    spot_radius=radius,
    theta=widgets.FloatSlider(min=0.0, max=np.pi*2, step=.01),
    tile_size=widgets.IntSlider(min=round(radius), max=round(radius*3), value=round(radius*2), step=1),
)

interactive(children=(FloatSlider(value=42.91152793434358, description='spot_radius', max=128.73458380303074, …

<function __main__.draw_spot_grid(grid, spot_radius, theta, tile_size)>